In [1]:
import sys
sys.path.insert(0, '../src')

In [2]:
from typing import Union, List
from pathlib import Path
import re
from ipywidgets import interact, IntSlider, Layout


from public_lib import load_detections, DetectionRenderer
import PIL.Image

In [3]:
# Adjust these to the where-ever your detections and frames are stored.

# detections_root should point to a folder with the structure
# detections_root
# |-- PXX
# |   |--- PXX_YY.pkl
detections_root = Path('../detections')

# frame_root shout point to a folder with the structure
# frames_root
# |-- PXX
# |   |-- PXX_YY
# |   |   |-- frame_zzzzzzzzzz.jpg
frames_root = Path('../frames')

In [4]:
video_id = 'P01_09'
participant_id = video_id[:3]
video_detections = load_detections(detections_root / participant_id / (video_id + '.pkl'))
max_frame_idx = len(video_detections) - 1
max_frame_idx

214269

In [5]:
frame_idx = 2500
video_detections[frame_idx]

FrameDetections(video_id='P01_09', frame_number=2501, objects=[ObjectDetection(bbox=BBox(top_left_x=0.4583333432674408, top_left_y=0.77734375, bottom_right_x=0.5701754093170166, bottom_right_y=0.9765625), score=0.8887454867362976)], hands=[HandDetection(bbox=BBox(top_left_x=0.5087719559669495, top_left_y=0.78515625, bottom_right_x=0.640350878238678, bottom_right_y=0.9765625), score=0.9911034107208252, state=<HandState.PORTABLE_OBJECT: 3>, side=<HandSide.RIGHT: 1>, object_offset=FloatVector(x=0.04746267572045326, y=0.0014553650980815291)), HandDetection(bbox=BBox(top_left_x=0.44078946113586426, top_left_y=0.80078125, bottom_right_x=0.5460526347160339, bottom_right_y=0.98828125), score=0.11721242219209671, state=<HandState.PORTABLE_OBJECT: 3>, side=<HandSide.LEFT: 0>, object_offset=FloatVector(x=0.027577195316553116, y=-0.06960134208202362))])

In [6]:
class LazyFrameLoader:
    def __init__(self, path: Union[Path, str], frame_template: str = 'frame_{:010d}.jpg'):
        self.path = Path(path)
        self.frame_template = frame_template
        
    def __getitem__(self, idx: int) -> PIL.Image.Image:
        return PIL.Image.open(str(self.path / self.frame_template.format(idx + 1)))

In [7]:
frames = LazyFrameLoader(frames_root / participant_id / video_id)

In [8]:
renderer = DetectionRenderer(hand_threshold=0.1, object_threshold=0.1)

@interact(frame_idx=IntSlider(value=13403, min=0, max=max_frame_idx, layout=Layout(width='650px')))
def render_detections(frame_idx):
    return renderer.render_detections(frames[frame_idx], video_detections[frame_idx])

interactive(children=(IntSlider(value=13403, description='frame_idx', layout=Layout(width='650px'), max=214269…